# AlaTB example for ff03-star force field - parallel 
In this example we run OpenMM simulations of the alanine dipeptide in parralel taking advantage of functionalities of the `multiprocessing` library.

In [1]:
import timeit
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout
import multiprocessing as mp
import multiprocessing.pool as pool

### Read GMX files
Read topology and coordinates from GMX files

In [2]:
gro = GromacsGroFile('alaTB_ff03-star_tip3p_solv.gro')

In [3]:
top = GromacsTopFile('alaTB_ff03-star_tip3p.top', \
        periodicBoxVectors=gro.getPeriodicBoxVectors(),
        includeDir='.')

### Simulation workflow
Here we will start with a standard simulation workflow, by first energy minimizing, then running short NVT with constraints, followed by a short NPT run and a production run.

In [4]:
system = top.createSystem(nonbondedMethod=PME, \
        nonbondedCutoff=1*nanometer,
        constraints=HBonds)

#### Defining position restraints
We add position restraints in the heavy atoms using the ```CustomExternalForce``` function.

In [5]:
force = CustomExternalForce(\
            "k*((x - x0)^2 + (y - y0)^2 + (z - z0)^2)")
force.addPerParticleParameter("k")
force.addPerParticleParameter("x0")
force.addPerParticleParameter("y0")
force.addPerParticleParameter("z0")
for i, atom_crd in enumerate(gro.positions):
    k = 1000.0*kilojoules_per_mole/nanometer**2
    x0 = atom_crd[0]
    y0 = atom_crd[1]
    z0 = atom_crd[2]
    if (gro.atomNames[i][0] in ('C', 'N', 'O')) \
            and (gro.atomNames[i][0] not in ('OW')):
        force.addParticle(i,[k, x0, y0, z0])
    else:
        force.addParticle(i,[0., x0, y0, z0])
system.addForce(force)

5

Next we generate the integrator with a ```LangevinIntegrator``` object and pass it on to a ```Simulation``` object. 

In [6]:
integrator = LangevinIntegrator(300*kelvin, \
                        1/picosecond, \
                        0.002*picoseconds)

In [7]:
nvt_posre = Simulation(top.topology, system, \
                        integrator)

We use the gro file atomic positions as initial positions for the simulation.

In [8]:
nvt_posre.context.setPositions(gro.positions)

First, we run a quick energy minimization.

In [9]:
nvt_posre.minimizeEnergy()

We define the reporters for both time series of conformations and also for energies.

In [10]:
pdb_rep = PDBReporter('nvt_posre.pdb', 1000)
nvt_posre.reporters.append(pdb_rep)
state_rep = StateDataReporter(stdout, 100, \
                step=True, potentialEnergy=True, \
                temperature=True, volume=True, density=True)
nvt_posre.reporters.append(state_rep)

In [11]:
nvt_posre.step(1000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)","Density (g/mL)"
100,-32505.172768115124,53.44526754110993,20.679574978308164,0.9753715458022438
200,-31786.414955615124,98.20438131528641,20.679574978308164,0.9753715458022438
300,-31093.008705615124,126.86463203000042,20.679574978308164,0.9753715458022438
400,-30649.805580615124,159.25653731686808,20.679574978308164,0.9753715458022438
500,-30197.868080615124,181.38491979202982,20.679574978308164,0.9753715458022438
600,-29787.391518115124,197.61491367292646,20.679574978308164,0.9753715458022438
700,-29335.555580615124,208.429101126481,20.679574978308164,0.9753715458022438
800,-29161.680580615124,229.50879053901258,20.679574978308164,0.9753715458022438
900,-28873.336830615124,237.36349177187486,20.679574978308164,0.9753715458022438
1000,-28557.883705615124,239.21337631801376,20.679574978308164,0.9753715458022438


#### NPT run
Next we run the NPT simulation. First we remove the position restrains from the system, because we have already equilibrated the water.

In [12]:
nforces = len(system.getForces())
system.removeForce(nforces-1)

We then grab positions and velocities from the previous simulation.

In [13]:
state = nvt_posre.context.getState(getPositions=True, \
                    getVelocities=True)

For NPT we must define a barostat that keeps control of the pressure.

In [14]:
barostat = MonteCarloBarostat(1.0*bar, 300.0*kelvin, 25)

In [15]:
system.addForce(barostat)

5

The remaining steps are very similar to those in the NVT simulation with position restraints.

In [16]:
integrator = LangevinIntegrator(300*kelvin, \
                        10/picosecond, \
                        0.002*picoseconds)

In [17]:
npt = Simulation(top.topology, system, \
                        integrator)
pdb_rep = PDBReporter('npt.pdb', 1000)
nvt_posre.reporters.append(pdb_rep)

npt.context.setState(state)
npt.reporters.append(state_rep)
npt.step(1000)

100,-27688.646867753705,289.0632611460894,20.776633597594014,0.9708150706119014
200,-27256.141714304802,299.8143416888052,20.832577856753606,0.9682080226373396
300,-27248.981906197558,295.4028479202531,20.81908669802052,0.9688354395989727
400,-27058.778781197558,289.4380135165073,20.81908669802052,0.9688354395989727
500,-26943.196018529386,299.86389814222844,20.812864212847227,0.9691250952704176
600,-26790.90702829315,308.35283205168855,20.711700573456202,0.9738586622373125
700,-26924.040946583293,305.5213659472505,20.618257401856333,0.9782722477463026
800,-26980.119382681936,289.7105601732537,20.32995953480353,0.9921450644599505
900,-26783.79649462094,293.50677679219467,20.639462590941164,0.9772671611119728
1000,-26958.413407113578,294.28146943002315,20.652013846060587,0.9766732272927153


We save the coordinates of the final step in `xml` format.

In [18]:
npt.saveState("npt.xml")

### Production NVT
This is the part we are changing here. Instead of simply running a simulation we are generating inputs for two identical runs. Again, we first remove the last force added, in this case the barostat.

In [19]:
nforces = len(system.getForces())
system.removeForce(nforces-1)

Here is where things change. We are starting one of the runs from the coordinates corresponding to the `xml` file we saved (`npt.xml`), and the other one from the `State` class object we kept in `npt`. For the former we only need the coordinates of the `xml` file, which we read from `mdtraj`.

In [20]:
import mdtraj as md
traj = md.load_xml("npt.xml", top='alaTB_ff03-star_tip3p_solv.gro')
last_frame = traj[-1].xyz[0]

Now comes the `multiprocessing` stuff, which is fairly simple. We just need to open a `ThreadPool` for whichever number of processes we want (here it is fixed as the number of processors). Then we prepare the runs using the `prepare_runs` next

In [21]:
def prepare_runs(init):
    """
    Prepares runs for OpenMM
    
    Parameters
    ----------
    init : list
        List containing indexes and npt input for runs.
        init[0] : index
        init[1] : initial state or configuration
 
    Returns
    -------
    nvt : 
        Simulation object with all information to simply be run.
    
    """
    ind = init[0]
    npt = init[1]
    
    integrator = LangevinIntegrator(300*kelvin, \
                        10/picosecond, \
                        0.002*picoseconds)
    platform = Platform.getPlatformByName('CPU')
    
    
    nvt = Simulation(top.topology, system, \
                        integrator, platform)
    
    try:
        state = npt.context.getState(getPositions=True, \
                    getVelocities=True)   
        nvt.context.setState(state)
    except AttributeError as e:
        nvt.context.setPositions(npt)
        nvt.context.setVelocitiesToTemperature(300*kelvin)

    dcd_rep = DCDReporter('nvt%i.dcd'%ind, 1000)
    nvt.reporters.append(dcd_rep)
    state_rep = StateDataReporter("nvt%i.txt"%init[0], 100, \
                step=True, potentialEnergy=True, \
                temperature=True, volume=True, density=True)
    nvt.reporters.append(state_rep)

    return nvt

In [22]:
nproc = mp.cpu_count()

In [23]:
tpool = pool.ThreadPool(processes=nproc)
mpinput = [(i,x) for i,x in enumerate([npt, npt, last_frame, last_frame])]
result = tpool.map(prepare_runs, mpinput)
tpool.close()
tpool.join()

`result` contains the output, which here are two `Simulation` objects that we need to run. Again, this will be done using multiprocessing, and for this we must write a function with one input only.

In [24]:
def do_run(x):
    """
    Worker for running the simulations
    
    Parameters:
    ----------
    x : list
    
        x[0]: simtk.openmm.app.simulation.Simulation
        Simulation object from OpenMM
        
        x[1]: steps

    Returns:
    --------
    elapsed : float
        Elapsed time between beginning and end.
    
    """
    sim = x[0]
    steps = x[1]
    print (sim, steps)
    start = timeit.timeit()
    sim.step(steps)
    end = timeit.timeit()
    elapsed = end - start
    return elapsed

In [ ]:
tpool = pool.ThreadPool(processes=nproc)
elapsed = tpool.map(do_run, list(zip(result, [10000, 10000, 10000,10000])))
tpool.close()
tpool.join()

In [ ]:
elapsed